In [1]:
import tensorflow as tf
import os
import random
import numpy as np
import flags

import matplotlib
import matplotlib.pyplot as plt


FLAGS = tf.app.flags.FLAGS
           
            
def get_filename(data_set):
    labels = []
    filename_set = []
    
    with open(FLAGS.data_dir + '/label.txt') as f:
        for line in f:
            inner_list = [elt.strip() for elt in line.split(',')]
            labels += inner_list
        
        for i, label in enumerate(labels):
            list = os.listdir(FLAGS.data_dir + '/' + data_set + '/' + label)
            for filename in list:
                filename_set.append([i,FLAGS.data_dir + '/' + data_set + '/' + label + '/'
                                   +filename])
        random.shuffle(filename_set)
        return filename_set
    
def read_jpeg(filename):
    read_file = tf.read_file(filename)
    decoded_image = tf.image.decode_jpeg(read_file, channels=FLAGS.channel)
    resized_image = tf.image.resize_images(decoded_image, [FLAGS.img_h,FLAGS.img_w])
    resized_image = tf.cast(resized_image, tf.uint8)
    
    return resized_image



        



def convert_images(sess, data_set):
    filename_set = get_filename(data_set)
    with open('./data/'  + data_set + '_data.bin', 'wb') as f:
        for i in range(0, len(filename_set)):
            resized_image = read_jpeg(filename_set[i][1])

            try:
                image = sess.run(resized_image)
            except Exception as e:
                print(e.message) 
                continue
            print(i, filename_set[i][0], image.shape)
            f.write(chr(filename_set[i][0]).encode())
            f.write(image.data)

            
            
            
# train, eval 에서 이미지 다시 불러오기            
def get_data(data_set, batch_size):
    if data_set is 'train':
        return distorted_inputs(batch_size)
    else:
        return eval_inputs(batch_size)
    
    
def distorted_inputs(batch_size):
    image, label = read_raw_images('train')
    reshaped_image = tf.cast(image,tf.float32)
    
    distorted_image = tf.random_crop(reshaped_image, [FLAGS.height,FLAGS.width,FLAGS.channel])
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image, lower=0.2, upper=1.8)
    float_image = tf.image.per_image_standardization(distorted_image)
        
    return generate_image_and_label_batch(float_image, label, 100, batch_size, shuffle=True)
                                          
def eval_inputs(batch_size):
    image, label = read_raw_images('eval')
    reshaped_image = tf.cast(image, tf.float32)

    resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, FLAGS.height, FLAGS.width)
    return generate_image_and_label_batch(resized_image, label, 10, batch_size, shuffle=True)
                                          
                                          
def read_raw_images(data_set):
    filename = ['./data/'  + data_set + '_data.bin']
    filename_queue = tf.train.string_input_producer(filename)
    size = (FLAGS.img_h) * (FLAGS.img_w) * FLAGS.channel
    record_bytes = size + 1
    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    key, value = reader.read(filename_queue)
    record_bytes = tf.decode_raw(value, tf.uint8)
    
    label = tf.cast(tf.slice(record_bytes, [0],[1]), tf.int32)
    image = tf.reshape(tf.slice(record_bytes,[1],[size]),
                       [FLAGS.channel,FLAGS.img_h,FLAGS.img_w])
    image_ = tf.transpose(image, [1,2,0])
    
    return image_, label

def generate_image_and_label_batch(image, label, min_queue_examples, batch_size, shuffle):
    num_preprocess_threads = 4
    if shuffle:
        images, label_batch = tf.train.shuffle_batch(
            [image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples * batch_size,
            min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch(
            [image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples * batch_size)

    return images, tf.reshape(label_batch, [batch_size])

def main(argv = None):
    with tf.Session() as sess:
        convert_images(sess, 'eval')
        print("complete")
        convert_images(sess, 'train')        
        

if __name__ == '__main__' :
    tf.app.run()
    

C:\Users\ghost\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Invalid GIF data (size 12081), the first frame does not fill the canvas
	 [[Node: DecodeJpeg = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile)]]
	 [[Node: resize_images/ResizeBilinear/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_8_resize_images/ResizeBilinear", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
1 1 (144, 144, 3)
Invalid GIF data (size 9269), the first frame does not fill the canvas
	 [[Node: DecodeJpeg_2 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_2)]]
	 [[Node: resize_images_2/ResizeBilinear/_21 = _Recv[cli

57 1 (144, 144, 3)
58 1 (144, 144, 3)
59 1 (144, 144, 3)
60 1 (144, 144, 3)
Invalid GIF data (size 11713), failed to slurp gif file: Image is defective, decoding aborted
	 [[Node: DecodeJpeg_61 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_61)]]
62 1 (144, 144, 3)
63 1 (144, 144, 3)
64 1 (144, 144, 3)
65 1 (144, 144, 3)
Invalid GIF data (size 18484), failed to slurp gif file: Image is defective, decoding aborted
	 [[Node: DecodeJpeg_66 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_66)]]
	 [[Node: resize_images_66/ResizeBilinear/_533 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=

130 1 (144, 144, 3)
Invalid GIF data (size 17870), failed to slurp gif file: Image is defective, decoding aborted
	 [[Node: DecodeJpeg_131 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_131)]]
132 1 (144, 144, 3)
133 1 (144, 144, 3)
134 1 (144, 144, 3)
135 1 (144, 144, 3)
136 1 (144, 144, 3)
Invalid GIF data (size 10954), failed to slurp gif file: Failed to read from given file
	 [[Node: DecodeJpeg_137 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_137)]]
	 [[Node: DecodeJpeg_137/_1097 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_4_DecodeJpeg_137", tensor_typ

195 1 (144, 144, 3)
196 0 (144, 144, 3)
Invalid GIF data (size 17132), failed to slurp gif file: Failed to read from given file
	 [[Node: DecodeJpeg_197 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_197)]]
198 1 (144, 144, 3)
199 1 (144, 144, 3)
200 0 (144, 144, 3)
201 0 (144, 144, 3)
202 1 (144, 144, 3)
203 0 (144, 144, 3)
204 1 (144, 144, 3)
205 1 (144, 144, 3)
206 1 (144, 144, 3)
207 1 (144, 144, 3)
208 0 (144, 144, 3)
209 1 (144, 144, 3)
210 1 (144, 144, 3)
211 1 (144, 144, 3)
212 1 (144, 144, 3)
213 1 (144, 144, 3)
214 1 (144, 144, 3)
215 1 (144, 144, 3)
216 1 (144, 144, 3)
217 1 (144, 144, 3)
Invalid GIF data (size 7820), failed to slurp gif file: Image EOF detected before image complete
	 [[Node: DecodeJpeg_218 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device

...

1239 1 (144, 144, 3)
1240 1 (144, 144, 3)
1241 1 (144, 144, 3)
1242 1 (144, 144, 3)
Invalid GIF data (size 7623), failed to slurp gif file: Failed to read from given file
	 [[Node: DecodeJpeg_1243 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1243)]]
1244 1 (144, 144, 3)
1245 1 (144, 144, 3)
1246 1 (144, 144, 3)
1247 1 (144, 144, 3)
1248 0 (144, 144, 3)
Invalid GIF data (size 7404), failed to slurp gif file: Image is defective, decoding aborted
	 [[Node: DecodeJpeg_1249 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1249)]]
Invalid GIF data (size 20837), the first frame does not fill the canvas
	 [[Node: DecodeJpeg_1250 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, rati

1310 1 (144, 144, 3)
1311 1 (144, 144, 3)
1312 1 (144, 144, 3)
1313 1 (144, 144, 3)
1314 1 (144, 144, 3)
1315 0 (144, 144, 3)
1316 0 (144, 144, 3)
Invalid GIF data (size 4747), failed to slurp gif file: Failed to read from given file
	 [[Node: DecodeJpeg_1317 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1317)]]
1318 0 (144, 144, 3)
1319 1 (144, 144, 3)
Invalid GIF data (size 73), the first frame does not fill the canvas
	 [[Node: DecodeJpeg_1320 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1320)]]
1321 0 (144, 144, 3)
1322 1 (144, 144, 3)
1323 0 (144, 144, 3)
Invalid GIF data (size 3520), the first frame does not fill the canvas
	 [[Node: DecodeJpeg_1324 = DecodeJpeg[acceptable_fraction=1, channels=3, dct

1410 1 (144, 144, 3)
Got 8 frames, but animated gifs can only be decoded by tf.image.decode_gif or tf.image.decode_image
	 [[Node: DecodeJpeg_1411 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1411)]]
1412 1 (144, 144, 3)
Invalid GIF data (size 7195), the first frame does not fill the canvas
	 [[Node: DecodeJpeg_1413 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1413)]]
Invalid GIF data (size 13057), failed to slurp gif file: Image is defective, decoding aborted
	 [[Node: DecodeJpeg_1414 = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReadFile_1414)]]
	 [[Node: resize_images_141

1477 1 (144, 144, 3)
1478 1 (144, 144, 3)
1479 1 (144, 144, 3)
1480 1 (144, 144, 3)


KeyboardInterrupt: 